In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Path to dataset files

In [ ]:
PATH="../data/census"
TRAINING_SET="train.csv.bz2"
VALIDATION_SET="valid.csv.bz2"
TEST_SET="test.csv.bz2"
N_TRAIN_INSTANCES=None # replace this with None to load the whole training set
N_TEST_INSTANCES=None # replace this with None to load the whole test set

# Column names (i.e., features)

In [ ]:
colnames = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
           'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 
            'hours_per_week', 'native_country', 'income_greater_than_50k']

# Load dataset

In [ ]:
def load_dataset(path, 
                 dataset_filename, 
                 sep=",", 
                 header=None,
                 nrows=None,
                 names=colnames, 
                 index_col=False, 
                 na_values='?'):
    
    return pd.read_csv(path+"/"+dataset_filename, 
                       sep=sep, 
                       header=header,
                       nrows=nrows,
                       names=colnames, 
                       index_col=index_col, 
                       na_values='?')

In [ ]:
train = load_dataset(PATH, TRAINING_SET, nrows=N_TRAIN_INSTANCES)

In [ ]:
print("Shape of training set: {}".format(train.shape))

In [ ]:
train.head()

In [ ]:
test = load_dataset(PATH, TEST_SET, nrows=N_TEST_INSTANCES)

In [ ]:
print("Shape of test set: {}".format(test.shape))

In [ ]:
test.head()

# Preprocessing dataset

-  Remove any entry containing at least one NA

In [ ]:
def preprocessing_dataset(data):
    # remove education
    data = data.drop(['education'], axis=1)
    # Remove all the records containing at least one NAdata = data[~data.isnull().any(axis=1)]
    return data[~data.isnull().any(axis=1)]

In [ ]:
# preprocess training set
train = preprocessing_dataset(train)
print("Shape of training set: {}".format(train.shape))

In [ ]:
train.head()

In [ ]:
# preprocess test set
test = preprocessing_dataset(test)
print("Shape of test set: {}".format(test.shape))

# Hold out validation set from training set

In [ ]:
def split_dataset(data, label_col="income_greater_than_50k", test_size=0.1, random_state=42):
    
    X_train, X_test = train_test_split(data, 
                                       test_size=test_size, 
                                       random_state=random_state, 
                                       stratify=data[label_col])
    
    return X_train, X_test

In [ ]:
train, valid = split_dataset(train)

In [ ]:
print(train["income_greater_than_50k"].value_counts())
print(valid["income_greater_than_50k"].value_counts())
print(test["income_greater_than_50k"].value_counts())

# Feature Attacks

- ### age (_continuous_): 
```python 
None
```

- ### workclass (_Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked_):
```python
if workclass == "Never-worked": workclass = "Without-pay"
```

- ### fnlwgt (_continuous_):
```python 
None
```

- ### education (_Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool_): None
 
- ### education_num (_continuous_):
```python 
None
```
 
- ### marital_status (_Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse_):
```python
if marital_status == "Divorced" or marital_status == "Separated": marital_status = "Never-married"
```

- ### occupation (_Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces_): 
```python
if not occupation == "Other-service": occupation = "Other-service"
```

- ### relationship (_Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried_):
```python
None
```

- ### race (_White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black_):
```python 
None
```

- ### sex (_Female, Male_):
```python 
None
```

- ### capital_gain (_continuous_):
```python
capital_gain = capital_gain + 2500 (step = 500)
```

- ### capital_loss (_continuous_):
```python
None
```

- ### hours_per_week (_continuous_):
```python
hours_per_week = hours_per_week + 8 (step = 4)
```

- ### native_country (_United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands_):
```python 
None
```

# Perturbing dataset

Let $\mathcal{D} = \{(\mathbf{x_i}, y_i)\}_{i=\{1,\ldots, m\}}$ be the dataset of interest, where $\mathbf{x_i} \subseteq \mathbb{R}^n = (x_{i,1}, \ldots, x_{i,n})^T$.

Use the set of rules defined above to perturb every single instance $(\mathbf{x_i}, y_i)\in \mathcal{D}$.

In general, one could assume the attacker has a budget $b_i$ for perturbing the instance $(\mathbf{x_i}, y_i)$, therefore the overall budget $B$ of the attacker is $B = \sum_{i=1}^m b_i$. 

Moreover, each feature $x_{*,j}$ is associated with a cost $c_j$ which the attacker has to pay in order to perturb exactly the $j$-th feature.

In this first (simplified) attempt, we assume the attacker has full budget at its disposal for perturbing every instance, and the cost of perturbing an individual feature is the same for all features; in other words, $b_i = B~\forall i=1,\ldots,m$ and $c_j = C~\forall j=1,\dots,n$ (e.g., $C = 1$ _budget unit_). 

It turns out the _actual_ total budget available to the attacker is $m\cdot B$.

In [ ]:
def perturb_workclass(x):
    x_copy = x.copy()
    if x_copy['workclass'] == 'Never-worked':
        x_copy['workclass'] = 'Without-pay'
        
    return x_copy
    
def perturb_marital_status(x):
    x_copy = x.copy()
    if x_copy['marital_status'] == 'Divorced' or x_copy['marital_status'] == 'Separated':
        x_copy['marital_status'] = 'Never-married'
        
    return x_copy
    
def perturb_occupation(x):
    x_copy = x.copy()
    if x_copy['occupation'] != 'Other-service':
        x_copy['occupation'] = 'Other-service'
    
    return x_copy

def perturb_education_old(x):
    x_copy = x.copy()
    if x_copy['education'] == 'Doctorate':
        x_copy['education'] = 'Masters'
    else:
        if x_copy['education'] == 'Masters':
            x_copy['education'] = 'Bachelors'
        else:
            if x_copy['education'] == 'Bachelors':
                x_copy['education'] = 'HS-grad'
    
    return x_copy
    
def perturb_education_num(x):
    x_copy = x.copy()
    if x_copy['education_num']>1:
        x_copy['education_num'] -= 1
    return x_copy


def perturb_capital_gain(x):
    x_copy = x.copy()
    x_copy['capital_gain'] += 500
    
    return x_copy
    
# def perturb_capital_loss(x, step=500):
#     x_copy = x.copy()
#     x_copy['capital_loss'] -= 500

#     return x_copy
    
def perturb_hours_per_week(x):
    x_copy = x.copy()
    x_copy['hours_per_week'] += 4

    return x_copy

In [ ]:
def contains(ls, x):
    """
    Returns if a pandas.Series object (x) belongs to a list of pandas.Series objects (ls).

    This function takes as input list of pandas.Series objects (ls) and returns True
    if this contains the second input argument (i.e., the pandas.Series object x), False otherwise.
    
    Parameters
    ----------
    ls : list
        A list of pandas.Series objects

    x : pandas.Series
        A series object

    Returns
    -------
    bool

    """
    if not ls:
        return False
    for s in ls:
        if s.equals(x):
            return True
    return False

In [ ]:
def is_empty(queue):
    return len(queue) == 0

In [ ]:
def is_equal_perturbation(a, b):
    return a[0].equals(b[0]) and a[1] == b[1] and a[2] == b[2]
    

In [ ]:
def perturb_instance(x, x_id, features, budget, max_budget_per_feature, costs, skip_class=1):
    """
    Returns the set of possible perturbations of a given instance.

    This function takes as input an instance and returns a set of perturbations of that instance, 
    using the specified amount of budget and considering the cost of perturbing each individual feature.

    Parameters
    ----------
    x : pandas.Series
        The original instance
    x_id : int
        The instance identifier (will be the same for all perturbations associated with this instance)
    features : list
        The list of features which can be perturbed (i.e., those for which an attack rule is defined)
    budget : float
        The attacker's budget
    max_budget_per_feature : dict
        The maximum allowed amount of budget units that can be spend on each feature
    costs : dict
        A mapping between each feature and its cost of perturbation

    Returns
    -------
    pandas.DataFrame
        The set of perturbations (including the original instance, placed at the very beginning)
    float
        The residual budget available for the remaining instances of the original dataset
        (At this stage, we assume each instance will get the same amount of budget, 
        i.e., the attacker is given back the full initial budget for perturbing each instance)

    """
    
    # initialize the queue (FIFO) with both the original instance, the initial budget, and an empty dictionary of budget units spent so far
    queue = [(x, budget, {})]
    # visited perturbations
    seen = [(x, budget, {})]
    # initialize the set of perturbations of this instance with the empty list
    perturbations = []
    # caching already performed perturbations for a certain feature
    feature_perturbations = {}
    # save the inital budget
    initial_budget = budget
    
    # loop until the queue is not empty
    while not is_empty(queue):
        item = queue.pop() # dequeue the first inserted element
        x = item[0] # get the instance
        b = item[1] # get the residual budget
        budget_units_spent = item[2] # get the dictionary containing the amount of budget spent on each feature, so far
        
        #print("Dequeuing item from queue [current budget = {}]".format(b))
        
        # if the set of perturbations does not contain already x
        if not contains(perturbations, x):
            perturbations.append(x) # append the transformed instance to the list of perturbations (NOTE: the first one will be the original - i.e., non-perturbed - instance)

            
#         if x[-1]==skip_class:
#             continue
            
            
        # loop through all the features subject to the set of attack rules
        for f in features:
            # check if there is enough budget to perform an attack on feature f
            if costs[f] <= b and budget_units_spent.get(f, 0) + costs[f] <= max_budget_per_feature[f]: 
                # if so, just call off to the proper function, which will perturb x on f
                # and returns the list of all perturbations from x on f
                x_prime = globals()["perturb_"+str(f)](x)
                
                # the perturbation of x on f is inserted into the queue (enqueued) only if the attack rule applies
                # to check this, it is enough to test for equality between x_prime and x
                if not x_prime.equals(x): # i.e., the rule applies to the current instance
                    # only if x_prime != x (i.e., the rule applies) we can subtract from the current budget
                    # the value of the cost of perturbing f
                    updated_budget_units_spent = budget_units_spent.copy()
                    # update the number of budget units spend on feature f
                    if not f in updated_budget_units_spent:
                        updated_budget_units_spent[f] = costs[f]
                    else:
                        updated_budget_units_spent[f] += costs[f]

                    #print("Update budget units spent on feature [{}] [old budget units spent = {}; new budget units spent = {}]"
                    #      .format(f, budget_units_spent.get(f, 0), updated_budget_units_spent[f]))

                    #print("Enqueue the current perturbation [initial budget = {}; residual budget = {}]"
                    #      .format(initial_budget, b - costs[f]))
                    item = (x_prime, b - costs[f], updated_budget_units_spent)
                    if not any(item_prime for item_prime in seen if is_equal_perturbation(item, item_prime)):
                        queue.insert(0, item)
                        seen.append(item)
            
            #print()
        
        
    # print("Create the dataframe containing the computed perturbations of instance [ID = #{}]".format(x_id))
    perturbations_df = pd.DataFrame(perturbations)
    perturbations_df.insert(loc=0, 
                                column="instance_id", 
                                value=[x_id for i in range(perturbations_df.shape[0])], 
                                allow_duplicates=True)
    
    
    return perturbations_df, initial_budget

In [ ]:
def perturb_dataset(data, budget, max_budget_per_feature, costs, features):
    """
    Returns the dataset extended with all instance perturbations.

    This function takes as input a dataset and returns another dataset which is obtained from the original
    by adding all the possible perturbations an attacker with budget B can apply to every instance.

    Parameters
    ----------
    data : pandas.DataFrame
        The original dataset
    features : list
        The list of features which can be perturbed (i.e., those for which an attack rule is defined)
    budget : float
        The attacker's budget
    max_budget_per_feature : dict
        The maximum allowed amount of budget units that can be spend on each feature
    costs : dict
        A mapping between each feature and its cost of perturbation

    Returns
    -------
    pandas.DataFrame
        The perturbed dataset

    """
    # check if the dataset is valid
    if data is None or data.empty:
        print("***** No dataset available! *****\n")
        return # if not, just return None
    # prepare the perturbed dataset to be returned, initially empty with an extra "instance_id" column
    cols = ["instance_id"] + data.columns.tolist()
    perturbed_data = pd.DataFrame(columns=cols)
    
    # start with instance_id = 1
    instance_id = 1
    
    # loop through every single instance in the original dataset
    print("***** Loop through all the original instances... *****\n")
    for index, instance in data.iterrows():
        if instance_id%100==0:
            print("***** Trying to perturb instance [ID = #{}]... *****\n".format(instance_id))
        # check if there is still some budget to spend before trying to perturb the current instance
        #print("***** Checking if budget residual is positive... *****\n")
        
        if(budget):
            #print("***** Budget residual = {} *****\n".format(budget))
            # retrieve the set of perturbations obtained from the current instance, along with residual budget
            # this is provided by calling off to the `perturb_instance` function
            # print("***** Perturb instance [ID = #{}]... *****\n".format(instance_id))
            perturbations, budget = perturb_instance(instance, instance_id, features, budget, max_budget_per_feature, costs)
            # append the set of perturbations obtained from the current instance to the perturbed dataset
            #print("***** Append the latest perturbations to the final dataset *****\n")
            perturbed_data = perturbed_data.append(perturbations)
            # move to the next instance_id
            instance_id += 1
            
        # otherwise, the budget is exhausted 
        # (note that, at this stage, we are allowing the budget to be negative after the latest perturbation;
        # a smoother strategy would be to check the residual budget before actually committing the perturbations)
        else:
            print("***** No more budget available! [budget residual = {}] *****\n".format(budget))
            break
        
    # eventually, return the perturbed dataset
    print("***** Return the final perturbed dataset *****\n")
    return perturbed_data

In [ ]:
# list of "attackable" features
#features = ['workclass', 'marital_status', 'occupation', 'education']
features = ['workclass', 'marital_status', 'occupation', 'education_num', 'capital_gain', 'hours_per_week']

# dictionary containing the cost of perturbing each feature (in budget unit)
costs = {
    'workclass'     : 1,
    'marital_status': 1,
    'occupation'    : 1,
    'education_num' : 1, 
    'hours_per_week': 4,
    'capital_gain'  : 50
}

# dictionary containing the maximum allowed amount of budget units that can be spent on each feature
max_budget_per_feature = {
    'workclass'     : 1,
    'marital_status': 1,
    'occupation'    : 1,
    'education_num' : 2,
    'hours_per_week': 8,
    'capital_gain'  : 250
}

# Serialize both original (\_ori) and perturbed (\_att) datasets

# Invoke perturbation on the following datasets:

-  Training set
-  Validation set
-  Test set

In [ ]:
def serialize_dataset(p_data, 
                      path, 
                      dataset_filename, 
                      suffix, 
                      sep=",", 
                      compression="bz2", 
                      index=False):
    
    p_data.to_csv(path+"/"+dataset_filename.split(".")[0]+suffix, 
                  sep=sep, 
                  compression=compression, 
                  index=index)

In [ ]:
serialize_dataset(train, PATH, TRAINING_SET, "_ori.csv.bz2")
serialize_dataset(valid, PATH, VALIDATION_SET, "_ori.csv.bz2")
serialize_dataset(test, PATH, TEST_SET, "_ori.csv.bz2")

In [ ]:
# tutto
# 2 gain + il resto
# hours + education
# fino a education
B = [5, 15, 150, 300]

for budget in B:
    train_att = perturb_dataset(train, budget, max_budget_per_feature, costs, features)
    valid_att = perturb_dataset(valid, budget, max_budget_per_feature, costs, features)
    test_att = perturb_dataset(test, budget, max_budget_per_feature, costs, features)
    
    serialize_dataset(train_att, PATH, TRAINING_SET, "_B{}".format(budget)+".csv.bz2")
    serialize_dataset(valid_att, PATH, VALIDATION_SET, "_B{}".format(budget)+".csv.bz2")
    serialize_dataset(test_att, PATH, TEST_SET, "_B{}".format(budget)+".csv.bz2")